In [ ]:
import json
from collections import OrderedDict
from copy import deepcopy

import nemo.collections.asr as nemo_asr
import openpyxl
import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import Dataset
import torch.nn.init as init
from transformers import Wav2Vec2Model
import sys

from utils import *

## transducer

In [ ]:
import nemo.collections.asr as nemo_asr
# model = nemo_asr.models.EncDecRNNTModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")
model = nemo_asr.models.EncDecRNNTModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")

In [ ]:
a,b = model.transcribe(paths2audio_files = ["/home/ubuntu/elec823/clean.wav", "/home/ubuntu/elec823/clean2.wav"], return_hypotheses = True)

In [ ]:
print(a[0])

In [ ]:
print(a[1].text)

In [ ]:
t = a.word_confidence
print(t)

In [ ]:
type(t)

# Working on

In [ ]:
import nemo.collections.asr as nemo_asr

In [ ]:
from models import *
from utils import *
from my_loss import *

import nemo.collections.asr as nemo_asr
# model = nemo_asr.models.EncDecRNNTModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")
model = nemo_asr.models.EncDecRNNTModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")

MODEL = '01_mel_mono_freeze'

CONSTANTS = InitializationTrain(
    model_name=MODEL, 
    verbose=True,
    train_data_path='/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/',
    train_info_path='/home/ubuntu/elec823/clarity_CPC1_data/metadata/CPC1.train.json',
    train_audiogram_path='/home/ubuntu/elec823/clarity_CPC1_data/metadata/listeners.CPC1_train.json',
    train_listener_path='/home/ubuntu/elec823/clarity_CPC1_data/metadata/listener_data.CPC1_train.xlsx',
    log_path='/home/ubuntu/elec823/log/',
    save_path='/home/ubuntu/elec823/checkpoints/',
    orig_freq=32000,
    new_freq=16_000,
    seed=3407,
    device=None
    )
dataset = CPCdata(metadata=CONSTANTS.metadata)

kf = KFold(n_splits=10, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    train_loader = DataLoader(dataset=[dataset[i] for i in train_idx], batch_size=3, shuffle=True, pin_memory=True,num_workers=8)
    break
for speech_input, info_dict in train_loader:
    mono_path = [CONSTANTS.DATA_PATH + path + "_mono.wav" for path in info_dict['path']]
    break


In [ ]:
converted_path = ['/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S09844_L0231_E013_mono.wav',
 '/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S09928_L0219_E021_mono.wav',
 '/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S09463_L0209_E013_mono.wav']

In [ ]:
out = model.transcribe(paths2audio_files = converted_path, return_hypotheses = True)
# Use out[0]
# then will be a list of 'batch' hypotheses
# word confidence is out[0][i].word_confidence

In [ ]:
confidence = [out[0][i].word_confidence for i in range(len(out[0]))]

In [ ]:
len(confidence[0])

In [ ]:
t1 = confidence
def truncate_and_pad(tensor):
    FIXED_LENGTH = 10
    if type(tensor) is not torch.Tensor:
        tensor = torch.tensor(tensor)
    current_length = tensor.size(0)

    # If the tensor length is greater than the target length, truncate it
    if current_length > FIXED_LENGTH:
        tensor = tensor[:FIXED_LENGTH]
    # If the tensor length is less than the target length, pad it with zeros
    elif current_length < FIXED_LENGTH:
        pad_size = FIXED_LENGTH - current_length
        padding = torch.zeros(pad_size, dtype=tensor.dtype, device=tensor.device)
        tensor = torch.cat((tensor, padding), dim=0)
    return tensor

result = list(map(truncate_and_pad, t1))
print(result)
print(len(result))
print(len(result[0]))
print(result[1])

In [ ]:
a = torch.stack(result, dim=0)
print(a)
print(a.shape)

In [ ]:
m = nn.LSTM(input_size=10,
                    hidden_size=256,
                    num_layers=3,
                    batch_first=True,
                    bidirectional=True)
out, (h_n, c_n) = m(a)
print(out.shape)
m2 = nn.Linear(512, 1)
m2(out)
print(m)

In [ ]:
asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")

In [ ]:
confidence, _ = asr_model.transcribe(paths2audio_files = converted_path, return_hypotheses = True)
confidence = [confidence[i].word_confidence for i in range(len(confidence))]

In [ ]:
print(confidence.shape)

In [ ]:
confidence = torch.stack(list(map(truncate_and_pad, confidence)), dim=0)
print(confidence)

In [ ]:
bidirectional_lstm_predictor = nn.Sequential(
            nn.LSTM(input_size=10,
                    hidden_size=256,
                    num_layers=3,
                    batch_first=True,
                    bidirectional=True),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
            )
t1 = torch.randn(3, 10)
t2 = bidirectional_lstm_predictor(t1)
print(t2)
out = bidirectional_lstm_predictor(confidence)

## model

In [129]:
import json
from collections import OrderedDict
from copy import deepcopy

import nemo.collections.asr as nemo_asr
import openpyxl
import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import Dataset
import torch.nn.init as init
from transformers import Wav2Vec2Model
import sys

from utils import *

class WordConfidence(nn.Module):
    def __init__(self):
        super(WordConfidence, self).__init__()
        self.asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained("nvidia/stt_en_conformer_transducer_xlarge")
        self.bidirectional_lstm=nn.LSTM(
            input_size=10,
            hidden_size=256,
            num_layers=3,
            batch_first=True,
            bidirectional=True
            )
        self.predictor = nn.Sequential(
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, speech_input, meta_data, mono_path):
        # output of asr model is [B, word_len]
        confidence, _ = self.asr_model.transcribe(paths2audio_files = mono_path, return_hypotheses = True)
        confidence = [confidence[i].word_confidence for i in range(len(confidence))]
        # padding and truncating to 10: output shape [B, 10]
        confidence = torch.stack(list(map(self.truncate_and_pad, confidence)), dim=0)
        lstm_out, (h_n, c_n) = self.bidirectional_lstm(confidence)
        pred = self.predictor(lstm_out)
        
        return pred

        
    def truncate_and_pad(self, tensor):
        FIXED_LENGTH = 10
        if type(tensor) is not torch.Tensor:
            tensor = torch.tensor(tensor)
        current_length = tensor.size(0)

        # If the tensor length is greater than the target length, truncate it
        if current_length > FIXED_LENGTH:
            tensor = tensor[:FIXED_LENGTH]
        # If the tensor length is less than the target length, pad it with zeros
        elif current_length < FIXED_LENGTH:
            pad_size = FIXED_LENGTH - current_length
            padding = torch.zeros(pad_size, dtype=tensor.dtype, device=tensor.device)
            tensor = torch.cat((tensor, padding), dim=0)
        return tensor
model = WordConfidence()
converted_path = ['/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S09844_L0231_E013_mono.wav',
 '/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S09928_L0219_E021_mono.wav',
 '/home/ubuntu/elec823/clarity_CPC1_data/clarity_data/HA_outputs/train/S09463_L0209_E013_mono.wav']

[NeMo I 2023-03-27 03:48:50 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-03-27 03:48:52 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket1/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket2/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket3/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket4/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket5/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket6/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket7/tarred_audio_manifest.json
    - - /data/NeMo_ASR_SET/English/v3.0/train_bucketed/bucket8/tarred_audio_manifest.json
    sample_rate: 16000
    

[NeMo I 2023-03-27 03:48:52 features:286] PADDING: 0
[NeMo I 2023-03-27 03:48:56 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-03-27 03:49:06 save_restore_connector:247] Model EncDecRNNTBPEModel was successfully restored from /home/ubuntu/.cache/huggingface/hub/models--nvidia--stt_en_conformer_transducer_xlarge/snapshots/96472b7552a5d0559a22399ea300498c5412699f/stt_en_conformer_transducer_xlarge.nemo.


In [130]:
out = model(1,1,converted_path)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [131]:
print(out)

tensor([[0.4975],
        [0.4984],
        [0.4993]], grad_fn=<SigmoidBackward0>)
